# Network Expansion and Topology Modification

Demonstrates how to programmatically modify system topology by tapping existing lines and splitting buses. This is essential for planning studies where new substations or interconnections are evaluated.

Import the case and instantiate the `GridWorkBench`.

```python
from esapp import GridWorkBench
from esapp.grid import *
from esapp.saw._helpers import create_object_string

wb = GridWorkBench(case_path)
```

In [1]:
# This cell is hidden in the documentation.
# It performs the actual case loading for the example.
from esapp import GridWorkBench
from esapp.grid import *
from esapp.saw._helpers import create_object_string
import ast

with open('case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 3.0359 sec


## Tap Existing Transmission Lines

Select a branch to tap and insert a new bus at the midpoint. First, identify the branch parameters:

In [2]:
from esapp.saw._helpers import create_object_string

branches = wb[Branch]
b = branches.iloc[10]
tobus = b['BusNum']
frombus = b['BusNum:1']
circuit = b['LineCircuit']
branch_str = create_object_string("Branch", tobus, frombus, circuit)

Tap the transmission line at 50% of its length and create a new bus at the tap point:

In [ ]:
new_bus_num = int(wb[Bus, "BusNum"]["BusNum"].max()) + 100

wb.esa.TapTransmissionLine(
    branch_str, 
    50.0,  # 50% down the line
    new_bus_num,
    'CAPACITANCE',  # Shunt model type
    False, False, 
    'Tapped_Substation'
)

## Split a Bus

Split an existing bus into two buses connected by a tie-line:

In [ ]:
target_bus = 1
split_bus_num = int(wb[Bus, 'BusNum']['BusNum'].max()) + 1

wb.esa.SplitBus(
    create_object_string("Bus", target_bus), 
    split_bus_num, 
    insert_tie=True, 
    line_open=False, 
    branch_device_type="Breaker"
)

## Validate Network Changes

After modifying the network topology, validate the changes by solving the power flow on the modified system:

In [ ]:
V = wb.pflow()
V.tail()